# Coursera Capstone Project
## This notebook will be used primarily for the capstone project. ..




In [84]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 
import geocoder # import geocoder
import folium
from geopy.geocoders import Nominatim

In [85]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Week 3: Segmenting and Clustering Neighborhoods in Toronto

In [86]:
#grab URL and store it in a URL variable
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#obtain raw html
r = requests.get(URL)
#parse into beautifulsoup 
soup = BeautifulSoup(r.content) 
#find required content
table = soup.findAll('table',attrs ={'rules':'all','cellspacing':0,}) 
#convert to text
a = table[0].text

"""
beautiful soup text has a lot of newlines. split by new line, and then run through
the rest of the code to remove the newline and replace it with an empty char
"""
b = a.split("\n")
for word in b:
    word.replace("\n", "")
    word.replace("\n", "")
    word.replace("\n", "")
#create a list
c = []
#now only add stuff that isn't an empty char or does not contain Not Assigned
for word in b:
    if "Not assigned" not in word and word != "":
        c.append(word)
#create a new table to store results
table_contents=[]

for word in c:
    #dict to store values for each neighborhood
    cell = {}
    #Since each postal code is 3 digits, go till the third digit and set it as the postal code value
    cell['Postal Code'] = word[:3]
    #set a temp variable equal to the rest of the word, which has the borough and the neighborhood
    temp = word.replace(word[:3], '')
    #split by the parenthesis, and grab the first half which is the borough, set it to the borough of the cell
    cell['Borough'] = temp.split('(')[0]
    #same logic for the neighborhood, just strip all the uneeded characters
    cell['Neighborhood'] = (((((temp).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    #append the dict to the table
    table_contents.append(cell)
#convert to df
df=pd.DataFrame(table_contents)
#handle special cases
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

    
    
    

In [87]:
#print the shape 
df.shape

(103, 3)

In [88]:

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Couldn't get the geocoder package to work, so I used the Geospatial dataset to grab the Latitude and Longitude for each neighborhood


In [89]:
f = open("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv", "r")
df2 = pd.read_csv("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv",",",header = 0)
df2.head()




/Users/shreyasiyengar/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [90]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [91]:
result = pd.merge(df, df2[['Postal Code', 'Latitude', 'Longitude']], )
result

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [93]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trawno_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [96]:
map_trawno = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trawno)  
    
map_trawno

In [98]:
CLIENT_ID = 'KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG' # your Foursquare ID
CLIENT_SECRET = '4UAMZ5NN21XIQHRK0MRSJ20TYYMF5MABJLQAQEDLRLEPTWGA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG
CLIENT_SECRET:4UAMZ5NN21XIQHRK0MRSJ20TYYMF5MABJLQAQEDLRLEPTWGA
